# 서울시 편의점 매출 결정요인 분석
## 02. 탐색적 데이터 분석 (EDA)

---

### 이 노트북의 목표
회귀분석 전, 각 가설에 대한 사전 탐색을 통해:
1. 가설이 데이터에서 지지되는지 방향성 확인
2. 회귀분석 가정(선형성, 다중공선성 등) 사전 점검
3. 이상치나 데이터 품질 문제 파악

### 검토할 가설
- **H1**: 유동인구가 많을수록 매출이 높다
- **H2**: 점포수(밀집도)가 높을수록 매출이 높다
- **H3**: 상권유형에 따라 매출 차이가 있다

---

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# 데이터 로드
df = pd.read_csv('./분석데이터.csv', encoding='utf-8-sig')
print(f"데이터 로드: {len(df):,}건")

데이터 로드: 6,097건


## 1. 종속변수 탐색: 매출

회귀분석의 종속변수가 될 매출 데이터의 분포 특성 파악

In [2]:
sales = df['당월_매출_금액']

print("[매출 기초통계]")
print(f"  평균: {sales.mean()/1e8:.2f}억원")
print(f"  중앙값: {sales.median()/1e8:.2f}억원")
print(f"  표준편차: {sales.std()/1e8:.2f}억원")
print(f"  최소: {sales.min()/1e8:.2f}억원")
print(f"  최대: {sales.max()/1e8:.2f}억원")

print(f"\n[분포 특성]")
print(f"  왜도: {sales.skew():.2f}")
print(f"  첨도: {sales.kurtosis():.2f}")

if sales.skew() > 1:
    print("  → 오른쪽 꼬리가 긴 분포 (고매출 지역이 소수 존재)")
    print("  → 회귀분석 시 이상치 영향 주의 필요")

[매출 기초통계]
  평균: 27.11억원
  중앙값: 17.19억원
  표준편차: 32.01억원
  최소: 0.00억원
  최대: 351.91억원

[분포 특성]
  왜도: 3.92
  첨도: 23.46
  → 오른쪽 꼬리가 긴 분포 (고매출 지역이 소수 존재)
  → 회귀분석 시 이상치 영향 주의 필요


In [3]:
# 평균과 중앙값 차이 → 분포 비대칭성 확인
mean_median_ratio = sales.mean() / sales.median()
print(f"평균/중앙값 비율: {mean_median_ratio:.2f}")
print(f"→ 1보다 크면 고매출 이상치가 평균을 끌어올리는 것")

평균/중앙값 비율: 1.58
→ 1보다 크면 고매출 이상치가 평균을 끌어올리는 것


---
## 2. 가설 H1 탐색: 유동인구 → 매출

**"유동인구가 많을수록 매출이 높다"**

In [4]:
pop = df['총_유동인구_수']

print("[유동인구 기초통계]")
print(f"  평균: {pop.mean()/1e6:.2f}백만명")
print(f"  중앙값: {pop.median()/1e6:.2f}백만명")
print(f"  표준편차: {pop.std()/1e6:.2f}백만명")

[유동인구 기초통계]
  평균: 5.62백만명
  중앙값: 5.26백만명
  표준편차: 2.97백만명


In [5]:
# 상관관계 분석
corr, p_value = stats.pearsonr(df['총_유동인구_수'], df['당월_매출_금액'])

print("[H1 검토: 유동인구 vs 매출]")
print(f"  상관계수: {corr:.4f}")
print(f"  p-value: {p_value:.2e}")
print()

if corr > 0.7:
    strength = "강한 양의 상관"
elif corr > 0.4:
    strength = "중간 양의 상관"
elif corr > 0:
    strength = "약한 양의 상관"
else:
    strength = "음의 상관"

print(f"  해석: {strength}")
print(f"  → H1 {'지지됨' if corr > 0 and p_value < 0.05 else '지지되지 않음'} (예비 판단)")

[H1 검토: 유동인구 vs 매출]
  상관계수: 0.4991
  p-value: 0.00e+00

  해석: 중간 양의 상관
  → H1 지지됨 (예비 판단)


---
## 3. 가설 H2 탐색: 점포수 → 매출

**"점포수(밀집도)가 높을수록 매출이 높다"**

주의: 점포수가 많으면 총 매출이 높아지는 것은 당연할 수 있음.  
이는 "수요가 있는 곳에 점포가 많이 생긴다"는 역인과 가능성도 있음.

In [6]:
stores = df['점포_수']

print("[점포수 기초통계]")
print(f"  평균: {stores.mean():.1f}개")
print(f"  중앙값: {stores.median():.1f}개")
print(f"  표준편차: {stores.std():.1f}개")
print(f"  최소: {stores.min():.0f}개")
print(f"  최대: {stores.max():.0f}개")

[점포수 기초통계]
  평균: 6.2개
  중앙값: 5.0개
  표준편차: 5.5개
  최소: 0개
  최대: 51개


In [7]:
# 상관관계 분석
corr, p_value = stats.pearsonr(df['점포_수'], df['당월_매출_금액'])

print("[H2 검토: 점포수 vs 매출]")
print(f"  상관계수: {corr:.4f}")
print(f"  p-value: {p_value:.2e}")
print()

if corr > 0.7:
    strength = "강한 양의 상관"
elif corr > 0.4:
    strength = "중간 양의 상관"
elif corr > 0:
    strength = "약한 양의 상관"
else:
    strength = "음의 상관"

print(f"  해석: {strength}")
print(f"  → H2 {'지지됨' if corr > 0 and p_value < 0.05 else '지지되지 않음'} (예비 판단)")

[H2 검토: 점포수 vs 매출]
  상관계수: 0.8065
  p-value: 0.00e+00

  해석: 강한 양의 상관
  → H2 지지됨 (예비 판단)


---
## 4. 가설 H3 탐색: 상권유형 → 매출

**"상권유형에 따라 매출 차이가 있다"**

In [8]:
print("[상권유형별 평균 매출]")
print("-" * 40)

type_stats = df.groupby('주요_상권유형')['당월_매출_금액'].agg(['mean', 'median', 'count'])
type_stats['mean'] = type_stats['mean'] / 1e8
type_stats['median'] = type_stats['median'] / 1e8
type_stats = type_stats.sort_values('mean', ascending=False)

for idx, row in type_stats.iterrows():
    print(f"  {idx}: 평균 {row['mean']:.2f}억, 중앙값 {row['median']:.2f}억 (n={row['count']:.0f})")

[상권유형별 평균 매출]
----------------------------------------
  관광특구: 평균 82.61억, 중앙값 84.61억 (n=90)
  발달상권: 평균 62.63억, 중앙값 48.43억 (n=300)
  전통시장: 평균 28.53억, 중앙값 19.56억 (n=231)
  골목상권: 평균 24.91억, 중앙값 16.80억 (n=5137)
  미분류: 평균 13.28억, 중앙값 6.22억 (n=339)


In [9]:
# ANOVA 검정으로 그룹 간 차이 확인
groups = [group['당월_매출_금액'].values for name, group in df.groupby('주요_상권유형')]
f_stat, p_value = stats.f_oneway(*groups)

print("[H3 검토: 상권유형별 매출 차이 (ANOVA)]")
print(f"  F-통계량: {f_stat:.2f}")
print(f"  p-value: {p_value:.2e}")
print()

if p_value < 0.05:
    print(f"  → 상권유형별 매출 차이가 통계적으로 유의함")
    print(f"  → H3 지지됨 (예비 판단)")
else:
    print(f"  → 상권유형별 매출 차이가 유의하지 않음")

[H3 검토: 상권유형별 매출 차이 (ANOVA)]
  F-통계량: 206.46
  p-value: 2.46e-166

  → 상권유형별 매출 차이가 통계적으로 유의함
  → H3 지지됨 (예비 판단)


In [10]:
# 골목상권 대비 각 상권유형의 매출 차이
base_mean = df[df['주요_상권유형'] == '골목상권']['당월_매출_금액'].mean()

print("[골목상권 대비 매출 차이]")
print(f"기준: 골목상권 평균 {base_mean/1e8:.2f}억원")
print()

for t in ['발달상권', '전통시장', '관광특구', '미분류']:
    if t in df['주요_상권유형'].values:
        t_mean = df[df['주요_상권유형'] == t]['당월_매출_금액'].mean()
        diff = (t_mean - base_mean) / 1e8
        pct = (t_mean / base_mean - 1) * 100
        print(f"  {t}: {diff:+.2f}억원 ({pct:+.1f}%)")

[골목상권 대비 매출 차이]
기준: 골목상권 평균 24.91억원

  발달상권: +37.72억원 (+151.4%)
  전통시장: +3.62억원 (+14.5%)
  관광특구: +57.70억원 (+231.6%)
  미분류: -11.63억원 (-46.7%)


---
## 5. 회귀분석 가정 사전 점검

### 5.1 다중공선성 사전 점검

독립변수들 간의 상관관계가 높으면 회귀계수 해석이 어려워짐

In [11]:
# 연속형 독립변수 간 상관관계
corr_indep, p_indep = stats.pearsonr(df['점포_수'], df['총_유동인구_수'])

print("[독립변수 간 상관관계]")
print(f"  점포수 vs 유동인구: {corr_indep:.4f}")
print()

if abs(corr_indep) > 0.8:
    print("  ⚠️ 다중공선성 우려 (r > 0.8)")
    print("     → VIF 분석 필수")
elif abs(corr_indep) > 0.6:
    print("  ⚠️ 다중공선성 주의 (r > 0.6)")
    print("     → VIF 분석 권장")
else:
    print("  ✓ 다중공선성 문제 낮음")

[독립변수 간 상관관계]
  점포수 vs 유동인구: 0.4848

  ✓ 다중공선성 문제 낮음


### 5.2 이상치 현황

극단적 이상치가 회귀분석 결과를 왜곡할 수 있음

In [12]:
# IQR 기준 이상치 탐지
def count_outliers(series, name):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    n_outliers = ((series < lower) | (series > upper)).sum()
    pct = n_outliers / len(series) * 100
    print(f"  {name}: {n_outliers}개 ({pct:.1f}%)")

print("[IQR 기준 이상치 현황]")
count_outliers(df['당월_매출_금액'], '매출')
count_outliers(df['점포_수'], '점포수')
count_outliers(df['총_유동인구_수'], '유동인구')

[IQR 기준 이상치 현황]
  매출: 512개 (8.4%)
  점포수: 301개 (4.9%)
  유동인구: 192개 (3.1%)


---
## 6. EDA 종합 결론

In [13]:
# 상관계수 다시 계산
corr_h1, _ = stats.pearsonr(df['총_유동인구_수'], df['당월_매출_금액'])
corr_h2, _ = stats.pearsonr(df['점포_수'], df['당월_매출_금액'])
_, anova_p = stats.f_oneway(*[g['당월_매출_금액'].values for _, g in df.groupby('주요_상권유형')])

print("=" * 60)
print("EDA 종합 결론")
print("=" * 60)

print(f"""
[가설별 예비 검토 결과]

H1: 유동인구 → 매출
    상관계수 r = {corr_h1:.4f}
    → {'양의 상관 확인, 가설 지지 가능성 높음' if corr_h1 > 0.3 else '상관 약함, 추가 검토 필요'}

H2: 점포수 → 매출
    상관계수 r = {corr_h2:.4f}
    → {'강한 양의 상관, 가설 지지 가능성 높음' if corr_h2 > 0.5 else '추가 검토 필요'}

H3: 상권유형 → 매출
    ANOVA p-value = {anova_p:.2e}
    → {'그룹 간 유의한 차이 존재, 가설 지지 가능성 높음' if anova_p < 0.05 else '추가 검토 필요'}

[회귀분석 진행 여부]
    다중공선성: 점포수-유동인구 r = {corr_indep:.4f} ({'주의 필요' if abs(corr_indep) > 0.6 else '문제 없음'})
    이상치: 존재하나 분석 진행 가능
    
    → 회귀분석 진행 적합
""")

print("\n다음 단계: 03_시각화.ipynb에서 위 결과를 시각적으로 확인")

EDA 종합 결론

[가설별 예비 검토 결과]

H1: 유동인구 → 매출
    상관계수 r = 0.4991
    → 양의 상관 확인, 가설 지지 가능성 높음

H2: 점포수 → 매출
    상관계수 r = 0.8065
    → 강한 양의 상관, 가설 지지 가능성 높음

H3: 상권유형 → 매출
    ANOVA p-value = 2.46e-166
    → 그룹 간 유의한 차이 존재, 가설 지지 가능성 높음

[회귀분석 진행 여부]
    다중공선성: 점포수-유동인구 r = 0.4848 (문제 없음)
    이상치: 존재하나 분석 진행 가능

    → 회귀분석 진행 적합


다음 단계: 03_시각화.ipynb에서 위 결과를 시각적으로 확인
